**Importing the libraries pandas and lxml install and geocoder**

In [ ]:
import pandas as pd
!pip install lxml
!pip install geocoder

**Imoprt the html data using pandas read_html**


In [ ]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

scraping the required table from wiki page

In [ ]:
df = data[0]
df.head()

Dropping all the rows where District is Not assigned

In [ ]:
df.drop(df.loc[df['Borough'] == 'Not assigned'].index, inplace=True)
df.head()

Reseting index and droping the index column

In [ ]:
df = df.reset_index()
df.drop(df.columns[0], 1, inplace=True)
df.head()

Checking if any value in Neighbourhood colunm is Not assigned

In [ ]:
df.loc[df.Neighbourhood == 'Not assigned']


Joining Neighbourhoods with same Postal code in the same District

In [ ]:
df = df.groupby(['Postal Code'], sort=False).agg(','.join)
df=df.reset_index()
df

In [ ]:
df.shape

In [ ]:
ltln_df = pd.read_csv('http://cocl.us/Geospatial_data')
ltln_df.head()

In [ ]:
df_to = pd.merge(left=df, right=ltln_df, left_on='Postal Code', right_on='Postal Code')
df_to

In [ ]:
df_to.shape

Foursquare credentials

In [ ]:
import requests
#CLIENT_ID =  # your Foursquare ID
#CLIENT_SECRET =  # your Foursquare Secret
#VERSION = '20180605' # Foursquare API version

#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)


Function to get nearby venue details from FS API about different borough in Toronto metropolitian

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
tor_venues = getNearbyVenues(names=df_to['Borough'],
                                   latitudes=df_to['Latitude'],
                                   longitudes=df_to['Longitude']
                                  )

In [ ]:
tor_venues.shape
tor_venues.head()

In [ ]:
tor_venues.groupby('Neighborhood').count()

In [ ]:
len(tor_venues['Venue Category'].unique())

One hot encoding of different venues in each neighborhood

In [ ]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
tor_venues.reset_index()
#tor_venues
tor_onehot['Neighborhood'] = tor_venues['Neighborhood']

first_column = tor_onehot.pop('Neighborhood')

tor_onehot.insert(0, 'Neighborhood', first_column )

tor_onehot.head()



In [ ]:
tor_onehot.shape

In [ ]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped.shape

Finding the Top 5 venues for each neighborhood

In [ ]:
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Ranking the most common venues in each Neighborhood

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Using Kmeans to cluster different neighborhoods based on similarities in types of venues

In [ ]:
!pip install sklearn
!pip install folium
from sklearn.cluster import KMeans
import folium # map rendering library

# set number of clusters
kclusters = 5

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Adding the cluster labels and joining the dataframes with the original dataframe

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = df_to

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Borough')

tor_merged.head() # check the last columns!

Plotting the Map of toronto and embedding different clusters predicted by Kmeans

In [ ]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[43.65, -79.34], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Borough'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters